<p style="font-family: Arial; font-size:2.75em;color:purple; font-style:bold"><br>
Random Forest using Python (sklearn):</p><br>
<p style="font-family: Arial; font-size:2.25em;color:green; font-style:bold"><br>
Kumar Rahul</p><br>

### We will be using Earnings Manipulation data (fraud_data.csv) in this exercise. Refer the Exhibit 1 to understand the feature list. Use the data and answer the below questions.

> 1.	Load the dataset in Jupyter Notebook using pandas
2.	Build a correlation matrix between all the numeric features in the dataset. Report the features, which are correlated at a cut-off of 0.70. What actions will you take on the features, which are highly correlated?
3.	Create a new data frame with the numeric features and categorical features as dummy variable coded features. Which features will you include for model building and why?
4.	Split the data into training set and test set. Use 80% of data for model training and 20% for model testing. 
5.	Use sklearn package to build a random forest model using `Status` as a dependent variable and all other features as independent variable. Report the model performance on the test set.
6.	Use sklearn model selection module to fine-tune the model parameters of random forest model. Report the model performance on the test set.


**PS: Not all the questions are being answered as a part of the same notebook. You are encouraged to answer the questions if you find them missing.**

**Exhibit 1**


|Sl. No.|	Name of Variable|	Variable Description|
|--------|--------------|------------------|
|1	|Company ID	|Unique Identifier|
|2 |	DSRI	|Days’ Sales in Receivables Index (DSRI): A large increase in receivable days might suggest accelerated revenue recognition to inflate profits.|
|3	|GMI	|Gross Margin Index (GMI): A deteriorating gross margin sends a negative signal about a firm’s prospects and creates an incentive to inflate profits.|
|4	|AQI	 |Asset Quality Index (AQI): An increase in long term assets (for example, the capitalisation of costs), other than property plant and equipment, relative to total assets indicates that a firm has potentially increased its involvement in cost deferral to inflate profits.|
|5	|SGI	|Sales Growth Index (SGI): High sales growth does not imply manipulation but high growth companies are more likely to commit financial fraud because their financial position and capital needs put pressure on managers to achieve earnings targets. If growth firms face large stock prices losses at the first indication of a slowdown, they may have greater incentives to manipulate earnings.|
|6	|DEPI 	 |Depreciation (DEPI): A falling level of depreciation relative to net fixed assets raises the possibility that a firm has revised upwards the estimated useful life of assets, or adopted a new method that is income increasing.|
|7	|SGAI	|Sales, General and Administrative Expenses (SGAI): Analysts might interpret a disproportionate increase in SG&A relative to sales as a negative signal about a firm’s future prospects, thereby creating an inventive to inflate profits.|
|8	|ACCR	|Accruals to Total Assets (ACCR): Total accruals are calculated as the change in working capital (other than cash) less depreciation relative to total assets. Accruals, or a portion thereof, reflect the extent to which managers make discretionary accounting choices to alter earnings. A higher level of accruals is, therefore, associated with a higher likelihood of profit manipulation.|
|9	|LEVI	|Leverage Index (LEVI): Leverage is measured as total debt relative to total assets. An increase in leverage creates an incentive to manipulate profits in order to meet debt covenants.|
|10|	Status	|Manipulator – Yes, Non Manipulator – No|

***

Learn more about random forest: https://www.stat.berkeley.edu/~breiman/RandomForests/cc_home.htm#intro

# Code starts here

To know the environment with the pyhton kernal



In [ ]:
import sys, os

sys.executable

Supress the warnings

In [ ]:
import warnings
warnings.filterwarnings("ignore")

We are going to use below mentioned libraries for **data import, processing and visulization**. As we progress, we will use other specific libraries for model building and evaluation. 

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sn # visualization library based on matplotlib
import matplotlib.pylab as plt

#the output of plotting commands is displayed inline within Jupyter notebook
%matplotlib inline 


## Data Import and Manipulation

### 1. Importing a data set

_Give the correct path to the data_



modify the ast_note_interactivity kernel option to see the value of multiple statements at once.

In [ ]:
import os

os.getcwd()

#os.chdir()

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
raw_df = pd.read_csv( "../Earnings_Manipulation_case/data/fraud_data.csv", 
                        sep = ',', na_values = ['', ' '])

raw_df.columns = raw_df.columns.str.lower().str.replace(' ', '_')
raw_df.head()

Dropping company_id as these will not be used for any analysis or model building.

In [ ]:
#?raw_df.drop()

In [ ]:
if set(['company_id']).issubset(raw_df.columns):
    raw_df.drop(['company_id'],axis=1, inplace=True)
    
raw_df.head()


### 2. Structure of the dataset



In [ ]:
raw_df.info()

In [ ]:
raw_df.status.value_counts()
#raw_df.describe(include='all').transpose()
raw_df.describe().transpose()

To get a help on the features of a object

In [ ]:
#?raw_df.status.value_counts()

### 2. Summarizing the dataset
Create a new data frame and store the raw data copy. This is being done to have a copy of the raw data intact for further manipulation if needed. The *dropna()* function is used for row wise deletion of missing value. The axis = 0 means row-wise, 1 means column wise.


In [ ]:
filter_df = raw_df.dropna(axis=0, how='any', thresh=None, 
                             subset=None, inplace=False)

list(filter_df.columns )

We will first start by printing the unique labels in categorical features

In [ ]:
numerical_features = ['dsri', 'gmi', 'aqi', 'sgi', 'depi', 'sgai', 'accr', 'levi']

categorical_features = ['status']

for f in categorical_features:
    print("\nThe unique labels in {} is {}\n".format(f, filter_df[f].unique()))
    print("The values in {} is \n{}\n".format(f,  filter_df[f].value_counts()))


We will use **groupby** function of pandas to get deeper insights on Manipulators **Yes** or **No**. We will write a generic function to report the mean by any categorical variable.

In [ ]:
def group_by (categorical_features):
    return filter_df.groupby(categorical_features).mean()



In [ ]:
group_by("status")

### 3. Visualizing the Data

Plot can be done using the callable functions of 

>1. pandas library (http://pandas.pydata.org/pandas-docs/stable/visualization.html)
2. matplotlib library (https://matplotlib.org/) or
3. seaborn library (https://seaborn.pydata.org/) which is based on matplotlib and provides interface for drawing attractive statistical graphics.

#### 3a. Visualizing the Data using seaborn

Write a custom function to create bar plot to visualize the average of numeric features w.r.t each categorical feature. Say, numeric features w.r.t status as status.

In [ ]:
def bar_plot(xlabel,ylabel,xcnt,ycnt):
    sn.barplot(x = xlabel, y = ylabel, data= filter_df, ax = axes[xcnt,ycnt])
    fig.show()

In [ ]:
numerical_features_set = ['dsri', 'gmi', 'aqi', 'sgi', 'depi', 'sgai', 'accr', 'levi']
categorical_features_set = ['status']

xcnt=0
ycnt = 0
fig, axes = plt.subplots(4,2, figsize=(10,12))
fig.subplots_adjust(hspace = 1, wspace=.5)

for c in categorical_features_set:
    for n in numerical_features_set:
        bar_plot(c,n,xcnt,ycnt)
        if ycnt <1:
            ycnt = ycnt+1
        else:
            ycnt = 0
            xcnt = xcnt+1

### Exercise

* Write a custom function to create boxplot plot to understand the outliers in the numeric features w.r.t status
* Write a custom function to create histogram plot to visualize the average of numeric features w.r.t status


## Model Building: 

### Dummy Variable coding

Remove the response variable from the dataset¶


In [ ]:
X_features = list(filter_df.columns)
X_features.remove('status')

In [ ]:
X_features

In [ ]:
categorical_features = ['status']

In [ ]:
encoded_X_df = pd.get_dummies( filter_df[X_features], drop_first = False )
encoded_Y_df = pd.get_dummies( filter_df['status'], drop_first=False)

In [ ]:
#?pd.get_dummies
encoded_Y_df.head()

In [ ]:
pd.options.display.max_columns = None
encoded_X_df.info()

In [ ]:
Y = encoded_Y_df.filter(['Yes'], axis =1)
X = encoded_X_df

### Train and test data split using Python

The train and test split can also be done using the **sklearn module**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size = 0.3, random_state = 42)

In [ ]:
y_test.Yes.value_counts()

y_train.Yes.value_counts()

## Model Building: Using the **sklearn** 



In [ ]:
from sklearn import ensemble #linear_model, ensemble, neural_network, naive bayes, svm, tree
#dir(ensemble)

In [ ]:
#?ensemble.GradientBoostingClassifier

In [ ]:
"""
The “balanced” mode uses the values of y to automatically adjust weights 
inversely proportional to class frequencies in the input data as 
n_samples / (n_classes * np.bincount(y))
"""

rf_model = ensemble.RandomForestClassifier()

rf_model.fit(X_train,y_train.values.ravel())
#rf_model.fit(os_data_X,os_data_y)

The problem with above approach is to understand which combination is the best. Should we have made a tree with `max_depth` as 4 and `min_sample_leaf` as 30 or should there have been other combination. Searching the grid for the best combination can be taken care of by `RandomizedSearchCV` method of `model_selection`

### Model with Random Search
Refer here to understand the details of parallel processing: https://stackoverflow.com/questions/32673579/scikit-learn-general-question-about-parallel-computing

#### Create search space

To use RandomizedSearchCV, create a parameter grid from where sample will be picked during model building:

In [ ]:
# Number of trees in random forest
n_estimators = [100,120,150,200, 240, 400, 500]

# Maximum number of levels in tree
max_depth = [2,3,4,5,6,7,8,9]
max_depth.append(None)

# Number of features to consider at every split
max_features = ['auto', 'log2']

# Minimum number of samples required to split a node
min_samples_split = [50, 75, 100]

# Minimum number of samples required at each leaf node
min_samples_leaf = [30, 35, 40]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# How to remove the class imbalance
class_weight=['balanced_subsample','balanced']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'class_weight': class_weight}
random_grid

To report the performance on the selected KPI use `sklearn.metrics.SCORERS.keys()` to get the list of all the metrics and pass the relevant one in `RandomizedSearchCV` or `GridSearchCV`

In [ ]:
from sklearn.metrics import SCORERS

SCORERS.keys()

Import RandomizedSearchCV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
#RandomizedSearchCV?

In [ ]:
# Use the random grid to search for best hyperparameters
from sklearn.model_selection import RandomizedSearchCV

rf_model = ensemble.RandomForestClassifier()

# Random search of parameters, using 3 fold cross validation, 
rf_best_model = RandomizedSearchCV(estimator = rf_model, 
                               param_distributions = random_grid, scoring = "precision",
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -2, pre_dispatch =2)
# Fit the random search model
rf_best_model
rf_best_model.fit(X_train, y_train.values.ravel())

### Report the parameter

The best model has the following parameter selected from the random search grid

In [ ]:
rf_best_model.best_params_

rf_best_model.best_estimator_

rf_best_model.best_score_

#rf__best_model.best_index_

### Exercise: Model with Grid Search

Random search allows to narrow down the range for each hyperparameter. Thus we know where to concentrate the search, to fine tune the model further. 

`GridSearchCV`, is a method that instead of sampling randomly from a distribution, evaluates all combinations which are defined. The grid search can be called `from sklearn.model_selection import GridSearchCV`

You are encouraged to fine tune the model further using Grid Search

## Model Evaluation

### 1. The prediction on train data.

To predict the outcome on the **train set**
> * Use **predict** function of the model object 


In [ ]:
predict_class_train_df = pd.DataFrame(rf_best_model.predict(X_train))
predict_class_train_df.head()

predict_porb_train_df = pd.DataFrame(rf_best_model.predict_proba(X_train))
predict_porb_train_df.iloc[:,:].head()

The above output clearly shows that the predcited class is the one for which the calculated probability is more compared to the calculated probability of the other class.

### 2. The prediction on test data.

The prediction can be carried out by **defining functions** as well. Below is one such instance wherein a function is defined and is used for prediction

In [ ]:
def get_predictions ( test_class, model, test_data ):
    predicted_df = pd.DataFrame(model.predict_proba(test_data))
    y_pred_df = pd.concat([test_class.reset_index(drop=True), predicted_df.iloc[:,1:]], axis =1)
    return y_pred_df

Giving label to the Y column of the test set by using the dictionary data type in python. This is being done for the model which was built using dummy variable coding. It will be used to generate confusion matrix at a later time

In [ ]:
y_test.Yes.value_counts()

In [ ]:
test_series = y_test
train_series = y_train

status_dict = {1:"Yes", 0:"No"}

y_test_df = test_series.replace(dict(Yes=status_dict))
y_test_df.rename({'Yes': 'status'}, axis='columns', inplace=True )

y_train_df = train_series.replace(dict(Yes=status_dict))

y_train_df.rename({'Yes': 'status'}, axis='columns', inplace=True )


In [ ]:
y_test_df.status.value_counts()

In [ ]:
predict_test_df = pd.DataFrame(get_predictions(y_test_df.status, rf_best_model, X_test))
predict_test_df.rename(columns = {1:'predicted_prob'}, inplace=True)
predict_test_df.head()

In [ ]:
predict_test_df['predicted'] = predict_test_df.predicted_prob.map(lambda x: 'Yes' if x > 0.5 else 'No')
predict_test_df[0:10]

### 3. Confusion Matrix

We will built classification matrix using the **metrics** method from **sklearn** package. We will also write a custom function to build a classification matrix and use it for reporting the performance measures.

To understand the concept of micro average and macro average:

https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin

#### 3a. Confusion Matrix using sklearn

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
print("The model with dummy variable coding output: ")
confusion_matrix(predict_test_df.status, predict_test_df.predicted)
rf_report = (classification_report(predict_test_df.status, predict_test_df.predicted))
print(rf_report)


#### 3b Confusion Matrix using generic function

In [ ]:
def draw_cm( actual, predicted ):
    plt.figure(figsize=(9,9))
    cm = metrics.confusion_matrix( actual, predicted )
    sn.heatmap(cm, annot=True,  fmt='.0f', xticklabels = ["No", "Yes"] , 
               yticklabels = ["No", "Yes"],cmap = 'Blues_r')
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Classification Matrix Plot', size = 15);
    plt.show()

The classification matrix plot as reported with dummy variable coding is:

In [ ]:
draw_cm( predict_test_df.status, predict_test_df.predicted )

### 4. Performance Measure on the test set


In [ ]:
def measure_performance (clasf_matrix):
    measure = pd.DataFrame({
                        'specificity': [round(clasf_matrix[0,0]/(clasf_matrix[0,0]+clasf_matrix[0,1]),2)], 
                        'sensitivity': [round(clasf_matrix[1,1]/(clasf_matrix[1,0]+clasf_matrix[1,1]),2)],
                        'recall': [round(clasf_matrix[1,1]/(clasf_matrix[1,0]+clasf_matrix[1,1]),2)],
                        'precision': [round(clasf_matrix[1,1]/(clasf_matrix[0,1]+clasf_matrix[1,1]),2)],
                        'overall_acc': [round((clasf_matrix[0,0]+clasf_matrix[1,1])/
                                              (clasf_matrix[0,0]+clasf_matrix[0,1]+clasf_matrix[1,0]+clasf_matrix[1,1]),2)]
                       })
    return measure

In [ ]:
cm = metrics.confusion_matrix(predict_test_df.status, predict_test_df.predicted)

lg_reg_metrics_df = pd.DataFrame(measure_performance(cm))
lg_reg_metrics_df

print( 'Total Accuracy sklearn: ',np.round( metrics.accuracy_score( predict_test_df.status, predict_test_df.predicted ), 2 ))




#### End of Document

***
***
